In [14]:
cd drive/MyDrive/Citadel Datathon 2023/data

/content/drive/MyDrive/Citadel Datathon 2023/data


In [15]:
ls APAC_2023_Datasets

 Crashes/  'Traffic, Investigations _ Other'/


In [16]:
ls APAC_2023_Datasets/Crashes

crash_info_commericial_vehicles.csv  crash_info_people.csv
crash_info_flag_variables.csv        crash_info_roadway.csv
crash_info_general.csv               crash_info_trailed_vehicles.csv
crash_info_motorcycle.csv            crash_info_vehicles.csv


In [17]:
ls "APAC_2023_Datasets/Traffic, Investigations _ Other"

crimes.csv                           police_districts.csv
hourly_weather_philadelphia.csv      police_stations.csv
investigations.csv                   traffic_stops_philadelphia.csv
philadelphia_population_metrics.csv


In [18]:
ls "Data Open 2023 Additional Data"

Boundaries_District.geojson  police_districts.geojson
philadelphia.geojson         Vital_Population_CT.geojson


In [19]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np

Our goal is to link all the data together using the police district number. 

Each row will represent the features of that police district (eg. car crash, crime) in a weekly/monthly basis

## Load related data

In [21]:
with open('Data Open 2023 Additional Data/philadelphia.geojson') as handle:
    philly_geo = json.loads(handle.read())
  
district_gdf = gpd.read_file('Data Open 2023 Additional Data/Boundaries_District.geojson')

## Police District

In [22]:
df_police_districts = pd.read_csv("APAC_2023_Datasets/Traffic, Investigations _ Other/police_districts.csv")
df_police_districts.head()

,OBJECTID,PERIMETER,DISTRICT_NUM,LOCATION,LAT,LNG,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
0,343,69282.58846,16,39th St. & Lancaster Ave.,39.96158,-75.19940,SWPD,1.216700e+08,1.927226e+07,27575.07918
1,344,33150.15496,17,20th St. & Federal St.,39.93728,-75.17670,SPD,5.786368e+07,9.154950e+06,13179.95335
2,345,54403.93004,18,55th St. & Pine St.,39.95426,-75.23217,SWPD,9.881929e+07,1.564117e+07,21629.61069
3,346,51597.05160,35,N. Broad St. & Champlost St.,40.04442,-75.14351,NWPD,1.544309e+08,2.450685e+07,20554.86914
4,347,58075.01444,39,22nd St. & Hunting Park Ave.,40.01362,-75.15884,NWPD,1.578976e+08,2.503787e+07,23123.58026


## Crime with district

In [23]:
crime = pd.read_csv("APAC_2023_Datasets/Traffic, Investigations _ Other/crimes.csv")

# create Geom column in crime_df using lat and lon
geometry = [Point(xy) for xy in zip(crime['lng'], crime['lat'])]
crime_gdf = gpd.GeoDataFrame(crime, crs="EPSG:4326", geometry=geometry)
crime_with_district = gpd.sjoin(crime_gdf, district_gdf, how='left', op='within')
crime_with_district = crime_with_district.dropna(subset=["DIST_NUM"])
print(len(crime_with_district))
crime_with_district.head()

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


1573848


,objectid,psa,dispatch_date,dispatch_time,location_block,text_general_code,lat,lng,fips,geometry,...,DISTRICT_ID,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
0,218891,2,2022-09-08,15:21:00,3200 BLOCK N BROAD ST,Thefts,40.002741,-75.153068,4.210102e+11,POINT (-75.15307 40.00274),...,NaN,39.0,NaN,39,22nd St. & Hunting Park Ave.,686-3390,NWPD,1.578976e+08,2.503787e+07,23123.580262
1,218892,2,2022-09-14,17:41:00,3200 BLOCK N BROAD ST,Thefts,40.002741,-75.153068,4.210102e+11,POINT (-75.15307 40.00274),...,NaN,39.0,NaN,39,22nd St. & Hunting Park Ave.,686-3390,NWPD,1.578976e+08,2.503787e+07,23123.580262
2,218893,2,2022-09-19,15:23:00,3200 BLOCK N BROAD ST,Thefts,40.002741,-75.153068,4.210102e+11,POINT (-75.15307 40.00274),...,NaN,39.0,NaN,39,22nd St. & Hunting Park Ave.,686-3390,NWPD,1.578976e+08,2.503787e+07,23123.580262
3,218894,2,2022-08-22,09:38:00,0 BLOCK WOLF ST,Theft from Vehicle,39.918351,-75.145999,4.210198e+11,POINT (-75.14600 39.91835),...,NaN,3.0,NaN,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793
4,218895,2,2022-08-20,10:31:00,0 BLOCK WOLF ST,Theft from Vehicle,39.918351,-75.145999,4.210198e+11,POINT (-75.14600 39.91835),...,NaN,3.0,NaN,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793


In [29]:
# We will keep the date and the types of crime (text_general_code)
crime_useful_feat = crime_with_district[["dispatch_date", "text_general_code", "DIST_NUM"]]
#crime_useful_feat = crime_useful_feat.join(pd.get_dummies(crime_with_district["text_general_code"]))
crime_useful_feat['dispatch_date'] = pd.to_datetime(crime_useful_feat['dispatch_date'])
crime_useful_feat['year_month'] = crime_useful_feat['dispatch_date'].dt.to_period('M')
crime_useful_feat = crime_useful_feat.drop("dispatch_date", axis=1)
crime_useful_feat.head()

<ipython-input-29-e88e3b80e6ca>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_useful_feat['dispatch_date'] = pd.to_datetime(crime_useful_feat['dispatch_date'])
<ipython-input-29-e88e3b80e6ca>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_useful_feat['year_month'] = crime_useful_feat['dispatch_date'].dt.to_period('M')


,text_general_code,DIST_NUM,year_month
0,Thefts,39.0,2022-09
1,Thefts,39.0,2022-09
2,Thefts,39.0,2022-09
3,Theft from Vehicle,3.0,2022-08
4,Theft from Vehicle,3.0,2022-08


In [57]:
crime_count = crime_useful_feat.groupby(["DIST_NUM", "year_month"]).count()

## Crash with District

In [30]:
df_crash = pd.read_csv("APAC_2023_Datasets/Crashes/crash_info_general.csv")

# create Geom column in crime_df using lat and lon
geometry = [Point(xy) for xy in zip(df_crash['DEC_LONG'], df_crash['DEC_LAT'])]
crash_gdf = gpd.GeoDataFrame(df_crash, crs="EPSG:4326", geometry=geometry)
crash_with_district = gpd.sjoin(crash_gdf, district_gdf, how='left', op='within')
crash_with_district = crash_with_district.dropna(subset=["DIST_NUM"])
print(len(crash_with_district))
crash_with_district.head()

<ipython-input-30-a4db084c09cf>:1: DtypeWarning: Columns (97,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df_crash = pd.read_csv("APAC_2023_Datasets/Crashes/crash_info_general.csv")
/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


131569


,Unnamed: 0,CRN,ARRIVAL_TM,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,BICYCLE_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_SUSP_SERIOUS_INJ_COUNT,BUS_COUNT,...,DISTRICT_ID,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
1,2,2010004457,658.0,2,0,0,0,0,0,0,...,NaN,6.0,NaN,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692
2,3,2010005005,1217.0,2,0,0,0,0,0,0,...,NaN,6.0,NaN,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692
4,5,2010005373,36.0,0,0,0,0,0,0,0,...,NaN,16.0,NaN,16,39th St. & Lancaster Ave.,686-3160,SWPD,1.216700e+08,1.927226e+07,27575.079183
5,6,2010005374,326.0,1,0,0,0,0,0,0,...,NaN,15.0,NaN,15,Harbison Ave. & Levick St.,686-3150,NEPD,3.077864e+08,4.881580e+07,35944.018281
6,7,2010005375,505.0,1,0,0,0,0,0,0,...,NaN,3.0,NaN,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793


In [31]:
# We will keep the following features
# categorical feat: *_TYPE, ILLUMINATION, MAX_SEVERITY_LEVEL, RELATION_TO_ROAD, ROAD_CONDITION, WEATHER1
# date feat: CRASH_MONTH, CRASH_YEAR, DATE_OF_WEEK

number_of_crashes = crash_with_district.groupby("DIST_NUM").count()["CRN"]
number_of_crashes


DIST_NUM
1.0      3333
2.0      6270
3.0      6475
5.0      1999
6.0      6995
7.0      4272
8.0      7706
9.0      4967
12.0     8005
14.0     5866
15.0    12130
16.0     8841
17.0     2537
18.0     7567
19.0     4887
22.0     6135
24.0     6893
25.0     7485
26.0     5183
35.0     7089
39.0     6890
77.0       44
Name: CRN, dtype: int64

In [32]:
#split crash_with_district into different year_month units
crash_with_district['date_str'] = crash_with_district['CRASH_YEAR'].astype(str) + '-' + crash_with_district['CRASH_MONTH'].astype(str).str.zfill(2)
crash_with_district['year_month'] = pd.to_datetime(crash_with_district['date_str']).dt.to_period('M')
crash_with_district = crash_with_district.drop('date_str', axis=1)

crash_with_district['year_month'] 

1         2010-01
2         2010-01
4         2010-01
5         2010-01
6         2010-01
           ...   
133008    2021-07
133009    2021-11
133010    2021-07
133011    2021-11
133012    2021-11
Name: year_month, Length: 131569, dtype: period[M]

In [48]:
manual_cat_list = ["ILLUMINATION", "ROAD_CONDITION", "WEATHER1"]
manual_drop_list = ["RDWY_SURF_TYPE_CD", "WORK_ZONE_TYPE","COLLISION_TYPE","TCD_TYPE"]
crash_useful_cat_feat_list = [i for i in crash_with_district.columns if ("_TYPE" in i and i not in manual_drop_list) or i in manual_cat_list]
crash_useful_num_feat_list = ["FATAL_COUNT","SUSP_SERIOUS_INJ_COUNT","SUSP_MINOR_INJ_COUNT"]
crash_useful_feat = crash_with_district[crash_useful_cat_feat_list+crash_useful_num_feat_list]
crash_useful_feat[["DIST_NUM","year_month"]]=crash_with_district[["DIST_NUM","year_month"]]

<ipython-input-48-b6ce2b439c26>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crash_useful_feat[["DIST_NUM","year_month"]]=crash_with_district[["DIST_NUM","year_month"]]
<ipython-input-48-b6ce2b439c26>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crash_useful_feat[["DIST_NUM","year_month"]]=crash_with_district[["DIST_NUM","year_month"]]


In [49]:
#compute the Traffic Hazard Impact Index
w1=10 #subjectively
w2=5
w3=2
crash_useful_feat["index"]=w1*crash_useful_feat["FATAL_COUNT"]+\
                    w2*crash_useful_feat["SUSP_SERIOUS_INJ_COUNT"]+w3*crash_useful_feat["SUSP_MINOR_INJ_COUNT"]+1
crash_useful_feat = crash_useful_feat.drop(["FATAL_COUNT","SUSP_SERIOUS_INJ_COUNT","SUSP_MINOR_INJ_COUNT"], axis=1)

<ipython-input-49-1edcfc7e9174>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crash_useful_feat["index"]=w1*crash_useful_feat["FATAL_COUNT"]+\


In [50]:
crash_useful_feat

,ILLUMINATION,INTERSECT_TYPE,LOCATION_TYPE,ROAD_CONDITION,WEATHER1,DIST_NUM,year_month,index
1,5.0,0,0,9,3,6.0,2010-01,3
2,1.0,0,0,1,3,6.0,2010-01,1
4,3.0,0,0,9,7,16.0,2010-01,1
5,3.0,3,2,9,3,15.0,2010-01,1
6,3.0,0,0,9,3,3.0,2010-01,1
...,...,...,...,...,...,...,...,...
133008,1.0,0,7,1,3,19.0,2021-07,1
133009,8.0,5,2,1,3,6.0,2021-11,8
133010,1.0,0,0,1,3,25.0,2021-07,1
133011,1.0,1,0,1,3,22.0,2021-11,1


## Traffic Stop Data

In [59]:
traffic_stop_with_district = pd.read_csv("APAC_2023_Datasets/Traffic, Investigations _ Other/traffic_stops_philadelphia.csv")
traffic_stop_with_district.head()

<ipython-input-59-9a81bfa2389d>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  traffic_stop_with_district = pd.read_csv("APAC_2023_Datasets/Traffic, Investigations _ Other/traffic_stops_philadelphia.csv")


,objectid,date,time,location,lat,lng,district,service_area,subject_age,subject_race,...,outcome,contraband_found,frisk_performed,search_conducted,search_person,search_vehicle,raw_race,raw_individual_contraband,raw_vehicle_contraband,fips
0,1,2014-01-01,01:14:00,NaN,NaN,NaN,19.0,191,31.0,black,...,arrest,True,False,True,True,False,Black - Non-Latino,True,False,NaN
1,2,2014-01-01,01:57:00,NaN,NaN,NaN,12.0,121,21.0,black,...,arrest,False,True,True,True,False,Black - Non-Latino,False,False,NaN
2,3,2014-01-01,03:30:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,24.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
3,4,2014-01-01,03:40:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,20.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
4,5,2014-01-01,08:30:00,N 56TH ST / UPLAND WAY,39.983712,-75.234188,19.0,193,31.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210101e+11


In [60]:
traffic_stop_with_district['date'] = pd.to_datetime(traffic_stop_with_district['date'])
traffic_stop_with_district['year_month'] = traffic_stop_with_district['date'].dt.to_period('M')

In [61]:
traffic_stop_with_district = traffic_stop_with_district.rename(columns={"district": "DIST_NUM"})
traffic_stop_count = pd.DataFrame(traffic_stop_with_district.groupby(["DIST_NUM", "year_month"]).count()['objectid'])

In [62]:
traffic_stop_count

objectid
DIST_NUM year_month          
1.0      2014-01          382
         2014-02         1132
         2014-03          993
         2014-04         1485
         2014-05         1419
...                       ...
77.0     2017-12           76
         2018-01           65
         2018-02           64
         2018-03           87
         2018-04           35

[1144 rows x 1 columns]

## Merge Crime, Crash, Traffic Stop

In [64]:
traffic_stop_count.reset_index()

,DIST_NUM,year_month,objectid
0,1.0,2014-01,382
1,1.0,2014-02,1132
2,1.0,2014-03,993
3,1.0,2014-04,1485
4,1.0,2014-05,1419
...,...,...,...
1139,77.0,2017-12,76
1140,77.0,2018-01,65
1141,77.0,2018-02,64
1142,77.0,2018-03,87


In [69]:
df_merged = crash_useful_feat.merge(traffic_stop_count.reset_index(), on=["DIST_NUM", "year_month"], how='left')
df_merged = df_merged.merge(crime_count.reset_index(), on=["DIST_NUM", "year_month"], how='left')

In [70]:
df_merged

,ILLUMINATION,INTERSECT_TYPE,LOCATION_TYPE,ROAD_CONDITION,WEATHER1,DIST_NUM,year_month,index,objectid,text_general_code
0,5.0,0,0,9,3,6.0,2010-01,3,NaN,NaN
1,1.0,0,0,1,3,6.0,2010-01,1,NaN,NaN
2,3.0,0,0,9,7,16.0,2010-01,1,NaN,NaN
3,3.0,3,2,9,3,15.0,2010-01,1,NaN,NaN
4,3.0,0,0,9,3,3.0,2010-01,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
131564,1.0,0,7,1,3,19.0,2021-07,1,NaN,743.0
131565,8.0,5,2,1,3,6.0,2021-11,8,NaN,579.0
131566,1.0,0,0,1,3,25.0,2021-07,1,NaN,668.0
131567,1.0,1,0,1,3,22.0,2021-11,1,NaN,850.0


In [73]:
df_merged = df_merged.rename({"objectid":"traffic_stop_count", "text_general_code": "crime_count"})
df_merged.to_csv("df_merged.csv")